<a href="https://colab.research.google.com/github/Kanishkirthik/SpamCall/blob/main/Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BI-DIRECTIONAL LSTM OUTPUT**

In [1]:
import numpy as np
import pandas as pd

In [2]:
df =pd.read_csv('sample_data/fraud_call.csv')

In [3]:
df.columns=["Label","Text"]
df=df.drop_duplicates()
df=df.dropna()

In [4]:
print(df.head())

    Label                                               Text
0   fraud  Todays Vodafone numbers ending with 4882 are s...
1  normal               Please don't say like that. Hi hi hi
2  normal                                         Thank you!
3  normal  Oh that was a forwarded message. I thought you...
4  normal  Got it. Seventeen pounds for seven hundred ml ...


In [5]:
df['Label'] = df['Label'].map({'fraud': 1, 'normal': 0})

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
texts = df["Text"].astype(str).tolist()
labels = df["Label"].tolist()

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [9]:
sequences = tokenizer.texts_to_sequences(texts)

In [10]:
maxlen=max([len(seq) for seq in sequences])

In [11]:
X=pad_sequences(sequences,maxlen=maxlen,padding='post')

In [12]:
y = np.array(labels)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,GRU,Layer,Flatten

In [14]:
print(df['Label'].value_counts())

Label
0    5014
1     596
Name: count, dtype: int64


In [15]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [16]:
from tensorflow.keras import layers,models

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

model = Sequential([
    # Embedding layer
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=maxlen),


    Bidirectional(LSTM(128, return_sequences=True)),  # LSTM layer with return_sequences=True for the next layer


    Dropout(0.2),


    Bidirectional(LSTM(64)),


    Dense(32, activation='relu'),


    Dropout(0.3),


    Dense(1, activation='sigmoid')
])

# Print the model summary to check the architecture
model.summary()




/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
import tensorflow as tf

# Define precision and recall metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# Define F1-score as a custom metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Compile the model with the new metrics
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', precision, recall, F1Score()]
)

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
xt, X_temp, yt, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Now split the temp set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [ ]:
print(yt)

[0 0 0 ... 0 0 0]


In [22]:
history = model.fit(
    xt, yt,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=10
)

Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.9137 - f1_score: 0.4461 - loss: 0.2639 - precision: 0.6887 - recall: 0.3487 - val_accuracy: 0.9875 - val_f1_score: 0.9412 - val_loss: 0.0533 - val_precision: 0.9492 - val_recall: 0.9333
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.9872 - f1_score: 0.9416 - loss: 0.0516 - precision: 0.9645 - recall: 0.9200 - val_accuracy: 0.9875 - val_f1_score: 0.9421 - val_loss: 0.0447 - val_precision: 0.9344 - val_recall: 0.9500
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.9954 - f1_score: 0.9777 - loss: 0.0167 - precision: 0.9826 - recall: 0.9731 - val_accuracy: 0.9822 - val_f1_score: 0.9180 - val_loss: 0.0622 - val_precision: 0.9032 - val_recall: 0.9333
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.9996 - f1_score: 0.9980 - loss: 0.0037 - precision: 0.9995 - recall: 0.9965 - val_accuracy: 0.9822 - val_f1_score: 0.9206 - val_loss: 0.0678 - val_precision: 0.8788 - val_rec

In [23]:
results1=model.evaluate(X_val,y_val)

18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 283ms/step - accuracy: 0.9860 - f1_score: 0.9185 - loss: 0.0623 - precision: 0.9873 - recall: 0.8624


In [24]:
print(f" validation Loss: {results1[0]}")
print(f"validation  Accuracy: {results1[1]}")
print(f" validation Precision: {results1[2]}")
print(f"validation  Recall: {results1[3]}")

 validation Loss: 0.10853464156389236
validation  Accuracy: 0.9839572310447693
 validation Precision: 0.9636363387107849
validation  Recall: 0.8833333253860474


In [25]:
results2=model.evaluate(X_test,y_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 342ms/step - accuracy: 0.9875 - f1_score: 0.9407 - loss: 0.1403 - precision: 0.9606 - recall: 0.9235


In [26]:
print(f"Test Loss: {results2[0]}")
print(f"Test Accuracy: {results2[1]}")
print(f"Test Precision: {results2[2]}")
print(f"Test Recall: {results2[3]}")

Test Loss: 0.147319495677948
Test Accuracy: 0.987522304058075
Test Precision: 0.9814814925193787
Test Recall: 0.8983050584793091


In [27]:
results3=model.evaluate(xt,yt)

141/141 ━━━━━━━━━━━━━━━━━━━━ 49s 345ms/step - accuracy: 0.9988 - f1_score: 0.9930 - loss: 0.0017 - precision: 0.9868 - recall: 1.0000


In [28]:
print(f"Train Loss: {results3[0]}")
print(f"Train Accuracy: {results3[1]}")
print(f"Train Precision: {results3[2]}")
print(f"Train Recall: {results3[3]}")

Train Loss: 0.0004135597846470773
Train Accuracy: 0.9997771978378296
Train Precision: 0.9979079365730286
Train Recall: 1.0


In [29]:
def classfiy(text):
  sequence=tokenizer.texts_to_sequences([text])
  padded_sequence=pad_sequences(sequence,maxlen=maxlen,padding='post')
  prediction=model.predict(padded_sequence,verbose=0)[0][0]
  print(prediction)
  if prediction>0.5:
    return 'fraud'
  else:
    return 'normal'

In [30]:
model.save("model.h5")

In [31]:
  print(classfiy('Todays Vodafone numbers ending with 4882 are selected to a receive a £350 award. If your number matches call 09064019014 to receive your £350 award'))

0.9999789
fraud


In [32]:
print(classfiy("Oh that was a forwarded message. I thought you send that to me."))

2.2469123e-07
normal
